# RFM用户价值模型（1）

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Step1：问题分析

|  RFM   | 含义  | 描述 |
|  :----:  | :----:  |  :----  |
| R（Recency）  | 最近一次消费时间 | 用户最近一次消费距离现在的时间 |
| F（Frequency） | 消费频率 | 用户在统计周期内购买商品的次数 |
| M（Monetary）| 消费金额 | 用户在统计周期内消费的总金额 |

## Step2：获取数据

In [2]:
filename = 'Online Retail Clean.xlsx'
sales_success = pd.read_excel(filename, sheet_name='Online Retail Clean')

### 时间一致化

In [3]:
# 列名重命名
sales_success.rename(columns={'InvoiceDate': 'InvoiceTime'}, inplace= True)
sales_success.dtypes

InvoiceNo                object
StockCode                object
Description              object
Quantity                  int64
InvoiceTime      datetime64[ns]
UnitPrice               float64
CustomerID                int64
Country                  object
UnitPrice_RMB           float64
SumPrice                float64
dtype: object

In [4]:
# 将InvoiceTime转为pandas能处理的时间格式datetime
sales_success['InvoiceTime'] = pd.to_datetime(sales_success['InvoiceTime'], errors='coerce')

In [5]:
# Date存放InvoiceTime中的日期部分
sales_success['Date'] = pd.to_datetime(sales_success['InvoiceTime'].dt.date, errors='coerce')
sales_success['Date'].dtypes

dtype('<M8[ns]')

In [6]:
# Month存放月份信息
sales_success['Month'] = sales_success['InvoiceTime'].astype('datetime64[M]')

In [7]:
# 星期几
sales_success['Dayofweek'] = sales_success['InvoiceTime'].dt.dayofweek

### 客户筛选

In [8]:
# 筛选CustomerID不为0的客户
sales_customer = sales_success[sales_success.CustomerID != 0].copy()
sales_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceTime,UnitPrice,CustomerID,Country,UnitPrice_RMB,SumPrice,Date,Month,Dayofweek
0,536365,85123A,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,22.3890,134.3340,2010-12-01,2010-12-01,2
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,29.7642,178.5852,2010-12-01,2010-12-01,2
2,536365,84406B,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,24.1450,193.1600,2010-12-01,2010-12-01,2
3,536365,84029G,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,29.7642,178.5852,2010-12-01,2010-12-01,2
4,536365,84029E,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,29.7642,178.5852,2010-12-01,2010-12-01,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
524873,581587,22613,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,7.4630,89.5560,2011-12-09,2011-12-01,4
524874,581587,22899,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,18.4380,110.6280,2011-12-09,2011-12-01,4
524875,581587,23254,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,36.4370,145.7480,2011-12-09,2011-12-01,4
524876,581587,23255,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,36.4370,145.7480,2011-12-09,2011-12-01,4


## Step5：数据建模

### 对客户进行分组

+ 通过两次Groupby分组计算订单次数

In [9]:
customer_grouped = sales_customer.groupby(['CustomerID', 'InvoiceNo'], as_index=False).agg({
    'SumPrice':sum,
    'Date':max
})
customer_grouped

,CustomerID,InvoiceNo,SumPrice,Date
0,12346,541431,677672.0080,2011-01-18
1,12347,537626,6249.5162,2010-12-07
2,12347,542237,4173.9242,2011-01-26
3,12347,549222,5586.2750,2011-04-07
4,12347,556201,3358.5256,2011-06-09
...,...,...,...,...
18527,18283,579673,1934.3218,2011-11-30
18528,18283,580872,1826.2400,2011-12-06
18529,18287,554065,6719.1584,2011-05-22
18530,18287,570715,8791.5896,2011-10-12


In [10]:
customer_grouped = customer_grouped.groupby('CustomerID').agg({
    'SumPrice':np.sum,
    'Date':np.max,
    'InvoiceNo':np.size
})
customer_grouped

,SumPrice,Date,InvoiceNo
CustomerID,,,
12346,677672.0080,2011-01-18,1
12347,37841.8000,2011-12-07,7
12348,15779.7672,2011-09-25,4
12349,15431.2890,2011-11-21,1
12350,2936.0320,2011-02-02,1
...,...,...,...
18280,1585.6680,2011-03-07,1
18281,709.5996,2011-06-12,1
18282,1563.2790,2011-12-02,2


### RFM值计算

In [11]:
rfm = pd.DataFrame()

#### R值

R（Recency）：最近一次消费时间，用户最近一次消费距离现在的时间。

In [12]:
now = pd.to_datetime('2011-12-10')
rfm['R'] = (now - customer_grouped['Date']).dt.days
rfm

,R
CustomerID,
12346,326
12347,3
12348,76
12349,19
12350,311
...,...
18280,278
18281,181
18282,8


#### F值

F（Frequency）：消费频率，用户在统计周期内购买商品的次数。

In [13]:
rfm['F'] = customer_grouped['InvoiceNo']
rfm

,R,F
CustomerID,,
12346,326,1
12347,3,7
12348,76,4
12349,19,1
12350,311,1
...,...,...
18280,278,1
18281,181,1
18282,8,2


#### M值

M（Monetary）：消费金额，用户在统计周期内消费的总金额。 

In [14]:
rfm['M'] = customer_grouped['SumPrice']
rfm

,R,F,M
CustomerID,,,
12346,326,1,677672.0080
12347,3,7,37841.8000
12348,76,4,15779.7672
12349,19,1,15431.2890
12350,311,1,2936.0320
...,...,...,...
18280,278,1,1585.6680
18281,181,1,709.5996
18282,8,2,1563.2790


### RFM值评分

In [15]:
rfm_rule_score = pd.DataFrame()

#### 划分方法1：按规则划分

##### R值

In [16]:
# 划分规则
r_bins = [0,30,60,90,120,1000000]
r_labels = [5,4,3,2,1]
# 评分
rfm_rule_score['R'] = pd.cut(rfm['R'], bins = r_bins, labels = r_labels, right = False)
rfm_rule_score.head()

,R
CustomerID,
12346,1
12347,5
12348,3
12349,5
12350,1


##### F值

In [17]:
# 划分规则
f_bins = [1,2,3,4,5,1000000]
f_labels = [1,2,3,4,5]
# 评分
rfm_rule_score['F'] = pd.cut(rfm['F'], bins = f_bins, labels = f_labels, right = False)
rfm_rule_score.head()

,R,F
CustomerID,,
12346,1,1
12347,5,5
12348,3,4
12349,5,1
12350,1,1


##### M值

In [18]:
# 划分规则
m_bins = [0,50,100,150,200,1000000]
m_labels = [1,2,3,4,5]
# 评分
rfm_rule_score['M'] = pd.cut(rfm['M'], bins = m_bins, labels = m_labels)
rfm_rule_score.head()

,R,F,M
CustomerID,,,
12346,1,1,5
12347,5,5,5
12348,3,4,5
12349,5,1,5
12350,1,1,5


In [19]:
rfm_rule_score.dtypes

R    category
F    category
M    category
dtype: object

In [20]:
# 修改格式
rfm_rule_score['R'] = rfm_rule_score['R'].astype(float)
rfm_rule_score['F'] = rfm_rule_score['F'].astype(float)
rfm_rule_score['M'] = rfm_rule_score['M'].astype(float)
rfm_rule_score.head()

,R,F,M
CustomerID,,,
12346,1.0,1.0,5.0
12347,5.0,5.0,5.0
12348,3.0,4.0,5.0
12349,5.0,1.0,5.0
12350,1.0,1.0,5.0


##### 总分

In [21]:
rfm_rule_score['Score'] = rfm_rule_score.apply(lambda x: x.sum(), axis=1)

In [22]:
rfm_rule_score

,R,F,M,Score
CustomerID,,,,
12346,1.0,1.0,5.0,7.0
12347,5.0,5.0,5.0,15.0
12348,3.0,4.0,5.0,12.0
12349,5.0,1.0,5.0,11.0
12350,1.0,1.0,5.0,7.0
...,...,...,...,...
18280,1.0,1.0,5.0,7.0
18281,1.0,1.0,5.0,7.0
18282,5.0,2.0,5.0,12.0


##### RFM统计

+ 找到RFM Score最大的用户

In [23]:
rfm_rule_score.sort_values('Score', ascending=False)

,R,F,M,Score
CustomerID,,,,
14367,5.0,5.0,5.0,15.0
13650,5.0,5.0,5.0,15.0
16814,5.0,5.0,5.0,15.0
13590,5.0,5.0,5.0,15.0
15426,5.0,5.0,5.0,15.0
...,...,...,...,...
17956,1.0,1.0,3.0,5.0
15823,1.0,1.0,3.0,5.0
13307,1.0,1.0,3.0,5.0


#### 划分方法2：按层次划分

+ 1、当R/F/M得分>R/F/M平均值时，赋值为“1”，反之则赋值为“0”
+ 2、根据得分，添加用户分层类型标签

|  RFM   | R  | F | M |
|  :----:  | :----:  |  :----:  |  :----:  |
| 重点价值客户  | 1 | 1 | 1 |
| 重点发展客户 | 1 | 0 | 1 |
| 重点保持客户 | 0 | 1 | 1 |
| 重点挽留客户  | 0 | 0 | 1 |
| 一般价值客户 | 1 | 1 | 0 |
| 一般发展客户 | 1 | 0 | 0 |
| 一般保持客户  | 0 | 1 | 0 |
| 一般挽留客户 | 0 | 0 | 0 |

##### 计算RFM值

In [24]:
rfm_mean_score = pd.DataFrame()

In [25]:
r_mean = rfm['R'].mean()
f_mean = rfm['F'].mean()
m_mean = rfm['M'].mean()
rfm_mean_score['R'] = rfm['R'].map(lambda x: 1 if x < r_mean else 0) # 小于
rfm_mean_score['F'] = rfm['F'].map(lambda x: 1 if x > f_mean else 0)
rfm_mean_score['M'] = rfm['M'].map(lambda x: 1 if x > m_mean else 0)
rfm_mean_score.head()

,R,F,M
CustomerID,,,
12346,0,0,1
12347,1,1,1
12348,1,0,0
12349,1,0,0
12350,0,0,0


##### 打标签

In [26]:
rfm_mean_score['Score'] = (rfm_mean_score['R'] * 100) + (rfm_mean_score['F'] * 10) + (rfm_mean_score['M'] * 1)
rfm_mean_score.head()

,R,F,M,Score
CustomerID,,,,
12346,0,0,1,1
12347,1,1,1,111
12348,1,0,0,100
12349,1,0,0,100
12350,0,0,0,0


|  RFM   | R  | F | M |
|  :----:  | :----:  |  :----:  |  :----:  |
| 重点价值客户  | 1 | 1 | 1 |
| 重点发展客户 | 1 | 0 | 1 |
| 重点保持客户 | 0 | 1 | 1 |
| 重点挽留客户  | 0 | 0 | 1 |
| 一般价值客户 | 1 | 1 | 0 |
| 一般发展客户 | 1 | 0 | 0 |
| 一般保持客户  | 0 | 1 | 0 |
| 一般挽留客户 | 0 | 0 | 0 |

In [27]:
#判断R/F/M是否大于均值
def transform_label(x):
    if x == 111:
        label = '重要价值客户'
    elif x == 11:
        label = '重要保持客户'
    elif x == 101:
        label = '重要发展客户'
    elif x == 1:
        label = '重要挽留客户'
    elif x == 110:
        label = '一般价值客户'
    elif x == 100:
        label = '一般发展客户'
    elif x == 10:
        label = '一般保持客户'
    elif x == 0:
        label = '一般挽留客户'
    return label

In [28]:
rfm_mean_score['Pattern'] = rfm_mean_score['Score'].apply(transform_label)
rfm_mean_score

,R,F,M,Score,Pattern
CustomerID,,,,,
12346,0,0,1,1,重要挽留客户
12347,1,1,1,111,重要价值客户
12348,1,0,0,100,一般发展客户
12349,1,0,0,100,一般发展客户
12350,0,0,0,0,一般挽留客户
...,...,...,...,...,...
18280,0,0,0,0,一般挽留客户
18281,0,0,0,0,一般挽留客户
18282,1,0,0,100,一般发展客户


#### RFM分层统计

In [29]:
rfm = rfm.merge(rfm_mean_score['Pattern'], on='CustomerID')

In [30]:
rfm

,R,F,M,Pattern
CustomerID,,,,
12346,326,1,677672.0080,重要挽留客户
12347,3,7,37841.8000,重要价值客户
12348,76,4,15779.7672,一般发展客户
12349,19,1,15431.2890,一般发展客户
12350,311,1,2936.0320,一般挽留客户
...,...,...,...,...
18280,278,1,1585.6680,一般挽留客户
18281,181,1,709.5996,一般挽留客户
18282,8,2,1563.2790,一般发展客户


+ 计算每种类型的顾客分别有多少人
+ 计算每种类型的顾客RFM均值的特征

In [31]:
rfm['Pattern'].value_counts()

一般发展客户    1750
一般挽留客户    1319
重要价值客户     689
一般价值客户     358
重要发展客户     117
一般保持客户      40
重要挽留客户      38
重要保持客户      27
Name: Pattern, dtype: int64

In [32]:
rfm.reset_index().groupby('Pattern').aggregate({
    'R':'mean',
    'F':'mean',
    'M':'mean',
    'CustomerID':'count'
})

,R,F,M,CustomerID
Pattern,,,,
一般价值客户,25.184358,6.522346,12107.768868,358
一般保持客户,160.875000,5.700000,9983.064135,40
一般发展客户,39.317714,2.121143,5589.006247,1750
一般挽留客户,220.306293,1.552691,3931.361357,1319
重要价值客户,19.518142,13.776488,72465.669725,689
重要保持客户,153.000000,8.814815,34678.899304,27
重要发展客户,38.213675,3.205128,43326.617222,117
重要挽留客户,178.973684,2.736842,63030.862378,38
